# Transfer learning practice

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Define directory paths
data_dir = 'AF_dataset'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')


In [ ]:
# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Pretrained VGG-16 mean and std
])

test_transforms = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)

# Data Loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:
# Load VGG-16 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier part
model.classifier[6] = nn.Linear(in_features=4096, out_features=2)
model = model.to(device)


Linear(in_features=4096, out_features=1000, bias=True)

In [5]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.classifier[6].parameters(), lr=0.001)

In [6]:
# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    return epoch_loss, accuracy


In [7]:
# Training loop
num_epochs = 10
best_accuracy = 0.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = evaluate(model, test_loader, criterion, device)
    
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Save the model if it has the best accuracy so far
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved Best Model!")


Epoch 1/10


100%|██████████| 6/6 [00:37<00:00,  6.20s/it]


Train Loss: 0.1549
Validation Loss: 0.0126
Validation Accuracy: 1.0000
Saved Best Model!
Epoch 2/10


100%|██████████| 6/6 [00:37<00:00,  6.17s/it]


Train Loss: 0.0304
Validation Loss: 0.0059
Validation Accuracy: 1.0000
Epoch 3/10


...

Train Loss: 0.0057
Validation Loss: 0.0026
Validation Accuracy: 1.0000
Epoch 9/10


100%|██████████| 6/6 [00:36<00:00,  6.12s/it]


Train Loss: 0.0093
Validation Loss: 0.0024
Validation Accuracy: 1.0000
Epoch 10/10


100%|██████████| 6/6 [00:36<00:00,  6.04s/it]

Train Loss: 0.0064
Validation Loss: 0.0018
Validation Accuracy: 1.0000


In [8]:
# Load the best model saved during training (optional)
model.load_state_dict(torch.load("best_model.pth"))

# Evaluate on the test set
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


/var/folders/b9/t4lk8c451h5dmb3m5570wjvc0000gn/T/ipykernel_49273/596005923.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.p

Test Loss: 0.0126
Test Accuracy: 1.0000


In [14]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Define path to the validation images
validation_dir = 'AF_dataset/validation'

# Define transformations (same as used during training)
val_transforms = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom Dataset for unlabelled images
class ValidationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")  # Ensure image is in RGB format
        if self.transform:
            image = self.transform(image)
        return image, img_name  # Return image and filename

# Initialize the validation dataset and dataloader
validation_dataset = ValidationDataset(root_dir=validation_dir, transform=val_transforms)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)


In [10]:
# Load the best model (ensure device is set to 'mps' if using Apple Silicon GPU)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model = model.to(device)
model.eval()

# Prepare to store predictions
predictions = []

# Perform inference on the validation set
with torch.no_grad():
    for images, filenames in validation_loader:
        images = images.to(device)
        outputs = model(images)
        
        # Get predicted class (1 for penguin, 2 for turtle)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().item() + 1  # Convert to 1-based class_id (1=Penguin, 2=Turtle)
        
        # Append the results in the format (filename, class_id)
        for filename in filenames:
            predictions.append((filename, preds))


/var/folders/b9/t4lk8c451h5dmb3m5570wjvc0000gn/T/ipykernel_49273/2241060273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.

In [11]:
import pandas as pd

# Convert predictions to DataFrame
submission_df = pd.DataFrame(predictions, columns=["filename", "class_id"])

# Save to CSV
submission_df.to_csv("submission_TL.csv", index=False)

print("Submission file created successfully as 'submission.csv'")


Submission file created successfully as 'submission.csv'


In [13]:
import pandas as pd

# Expected order of filenames
expected_filenames = [
	'image_id_004_.jpg',
	'image_id_010_.jpg',
	'image_id_016_.jpg',
	'image_id_024_.jpg',
	'image_id_033_.jpg',
	'image_id_034_.jpg',
	'image_id_037_.jpg',
	'image_id_038_.jpg',
	'image_id_043_.jpg',
	'image_id_050_.jpg',
	'image_id_054_.jpg',
	'image_id_066_.jpg',
	'image_id_073_.jpg',
	'image_id_075_.jpg',
	'image_id_076_.jpg',
	'image_id_086_.jpg',
	'image_id_090_.jpg',
	'image_id_092_.jpg',
	'image_id_106_.jpg',
	'image_id_110_.jpg',
	'image_id_112_.jpg',
	'image_id_115_.jpg',
	'image_id_122_.jpg',
	'image_id_124_.jpg',
	'image_id_129_.jpg',
	'image_id_154_.jpg',
	'image_id_156_.jpg',
	'image_id_174_.jpg',
	'image_id_177_.jpg',
	'image_id_180_.jpg',
	'image_id_183_.jpg',
	'image_id_185_.jpg',
	'image_id_187_.jpg',
	'image_id_189_.jpg',
	'image_id_190_.jpg',
	'image_id_193_.jpg',
	'image_id_208_.jpg',
	'image_id_212_.jpg',
	'image_id_221_.jpg',
	'image_id_223_.jpg',
	'image_id_224_.jpg',
	'image_id_228_.jpg',
	'image_id_239_.jpg',
	'image_id_247_.jpg',
	'image_id_250_.jpg',
	'image_id_263_.jpg',
	'image_id_264_.jpg',
	'image_id_265_.jpg',
	'image_id_268_.jpg',
	'image_id_273_.jpg',
	'image_id_287_.jpg',
	'image_id_304_.jpg',
	'image_id_307_.jpg',
	'image_id_310_.jpg',
	'image_id_312_.jpg',
	'image_id_315_.jpg',
	'image_id_328_.jpg',
	'image_id_329_.jpg',
	'image_id_337_.jpg',
	'image_id_339_.jpg',
	'image_id_344_.jpg',
	'image_id_348_.jpg',
	'image_id_352_.jpg',
	'image_id_357_.jpg',
	'image_id_365_.jpg',
	'image_id_369_.jpg',
	'image_id_370_.jpg',
	'image_id_377_.jpg',
	'image_id_388_.jpg',
	'image_id_389_.jpg',
	'image_id_390_.jpg',
	'image_id_394_.jpg',
	'image_id_403_.jpg',
	'image_id_410_.jpg',
	'image_id_419_.jpg',
	'image_id_430_.jpg',
	'image_id_434_.jpg',
	'image_id_442_.jpg',
	'image_id_445_.jpg',
	'image_id_458_.jpg',
	'image_id_462_.jpg',
	'image_id_471_.jpg',
	'image_id_474_.jpg',
	'image_id_477_.jpg',
	'image_id_492_.jpg',
	'image_id_495_.jpg',
	'image_id_498_.jpg',
	'image_id_500_.jpg',
	'image_id_514_.jpg',
	'image_id_519_.jpg',
	'image_id_524_.jpg',
	'image_id_527_.jpg',
	'image_id_537_.jpg',
	'image_id_545_.jpg',
	'image_id_548_.jpg',
	'image_id_551_.jpg',
	'image_id_563_.jpg',
	'image_id_571_.jpg',

]

# Load the generated submission DataFrame
submission_df = pd.read_csv("submission_TL.csv")

# Reorder based on expected filenames
submission_df["filename"] = pd.Categorical(submission_df["filename"], categories=expected_filenames, ordered=True)
submission_df = submission_df.sort_values("filename")

# Save the reordered submission file
submission_df.to_csv("submission_reordered.csv", index=False)
print("Reordered submission file saved as 'submission_reordered.csv'")


Reordered submission file saved as 'submission_reordered.csv'
